In [1]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
import tensorflow as tf
import pandas as pd

tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa",from_tf=True,)
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa",)


2022-03-22 21:31:05.661290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 21:31:05.661341: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-22 21:31:30.196802: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-22 21:31:30.196858: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-22 21:31:30.196909: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-03-22 21:31:30.197286: I tensorflow/core/platform/cpu_feature_gua

In [4]:
# emotion.save_pretrained("./EmoRoBERTa")

In [2]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train,y_train = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length",max_length=400,)#TODO increase size

In [23]:
inputs = tokenizer(X_train.to_list(), padding=True, truncation=True, return_tensors='tf',max_length =200,)
X_train_features ={'input_ids':inputs['input_ids'].numpy(),'input_attention':inputs['attention_mask'].numpy()}
X_train_labels=pd.get_dummies(y_train)
# # total_dataset = tf.data.Dataset.zip((dataset_features, dataset_labels))
# new_model = tf.keras.Sequential()
# new_model.add(model.layers[0])
# new_model.add(tf.keras.layers.Dense(13,activation = "softmax"))

In [92]:
new_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),loss=tf.keras.losses.BinaryCrossentropy(),metrics = [tf.keras.metrics.AUC()])

In [93]:
new_model.fit(x=X_train_features,y=X_train_features,batch_size=16,epochs = 4,workers = 4)

Epoch 1/4


ValueError: in user code:

    File "/home/anri/Desktop/Machine_Learning_project/ML_project/lib/python3.10/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/anri/Desktop/Machine_Learning_project/ML_project/lib/python3.10/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/anri/Desktop/Machine_Learning_project/ML_project/lib/python3.10/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/anri/Desktop/Machine_Learning_project/ML_project/lib/python3.10/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/anri/Desktop/Machine_Learning_project/ML_project/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_tr

In [28]:
model.save_weights('./saved_weights.h5')

{'input_ids': [[0, 100, 216, 141, 47, 619, 4, 1308, 471, 16, 562, 2682, 906, 8, 2682, 906, 6, 8, 24, 16, 45, 1531, 4, 2], [0, 6968, 8562, 4428, 98, 14083, 27785, 99, 10, 4045, 631, 7, 1394, 13, 2], [0, 4239, 20351, 16, 6344, 12846, 7686, 16, 2131, 6, 45473, 16, 3035, 4, 85, 18, 1531, 6, 3035, 8, 13350, 4, 33826, 966, 1782, 4, 38, 794, 24, 23, 5, 230, 5101, 2583, 4, 2], [0, 3084, 6, 53, 24, 473, 2770, 383, 19, 2396, 70, 110, 804, 5475, 11, 65, 317, 734, 9574, 14, 317, 16, 1204, 2], [0, 38356, 514, 6, 2225, 26073, 8, 1029, 1071, 31, 5, 1400, 53, 2653, 350, 4736, 7, 213, 120, 24, 2], [0, 250, 33130, 38, 802, 47, 18774, 162, 94, 363, 4, 38, 21, 38, 7533, 33, 127, 7409, 15, 127, 1028, 98, 95, 2788, 162, 11, 5, 23621, 15983, 116, 2], [0, 3983, 139, 734, 939, 21, 2818, 13, 10, 4486, 13058, 527, 19, 10, 326, 23999, 718, 2913, 4105, 3512, 1345, 2], [0, 118, 2813, 939, 115, 6396, 127, 2335, 141, 7, 310, 3023, 8304, 328, 40463, 734, 2362, 26081, 2], [0, 25607, 1295, 549, 7, 213, 7, 3267, 8, 386, 

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = np.argmax(model.predict(X_train),axis = 1)
print(classification_report(y_train, y_pred))

In [3]:
# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 200


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='max_length', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

In [4]:
input_ids_layer = tf.keras.layers.Input(shape=(200,), 
                                        name='input_ids', 
                                        dtype='int32')
input_attention_layer = tf.keras.layers.Input(shape=(200,), 
                                              name='input_attention', 
                                              dtype='int32')
last_hidden_state = model.layers[0](([input_ids_layer, input_attention_layer]))[0]
# cls = last_hidden_state[:,0,:]

In [10]:
last_hidden_state

<KerasTensor: shape=(None, 200, 768) dtype=float32 (created by layer 'roberta')>

In [5]:
one_more = tf.keras.layers.AveragePooling1D(1,200)(last_hidden_state)

In [6]:
pooling = tf.keras.layers.Reshape((768,))(one_more)
output = tf.keras.layers.Dense(13,activation='relu')(pooling)

In [7]:
model = tf.keras.Model([input_ids_layer, input_attention_layer], output)

In [8]:
model.output

<KerasTensor: shape=(None, 13) dtype=float32 (created by layer 'dense')>

In [9]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),loss=tf.keras.losses.BinaryCrossentropy(),metrics = [tf.keras.metrics.AUC()])

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 200)]        0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 200)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124055040   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 200,                                           

In [ ]:
model.fit(x=[X_train_ids,X_train_attention],y=pd.get_dummies(y_train),batch_size=2,epochs = 4,workers = 4)

Epoch 1/4


InvalidArgumentError: Graph execution error:

Detected at node 'assert_less_equal/Assert/AssertGuard/Assert' defined at (most recent call last):
    File "/usr/lib64/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib64/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/anry/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/anry/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/anry/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/anry/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib64/python3.10/asyncio/base_events.py", line 595, in run_forever
      sel